In [ ]:

#Import libary yang dibutuhkan bagi naive bayes
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
seed = 1234
from google.colab import drive #Import data dilakukan melalui cloud yaitu Google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/My Drive/Project Big Data/Axioo (Setelah format dirubah)(2).csv', sep=';', encoding='utf-8')
data.head()

,time,username,komen,rating,label
0,1 bulan lalu,rani,Terimakasih untuk saler Atas amanah nya Semoga...,5,Positif
1,1 bulan lalu,Sofia,"alhamdulillah sampai dengan selamat, seller ra...",5,Positif
2,5 bulan lalu,M,"hati2 yg mo beli, pastikan brg sesuai dg kesep...",1,Negatif
3,5 bulan lalu,Abdi,Barang sdh sampai sesuai dgn deskripsi. Penjua...,5,Positif
4,5 bulan lalu,Ridho,untuk sejauh ini berjalan normal semoga bisa a...,4,Positif


In [ ]:
#proses case folding : Pada bagian ini data akan disama ratakan menjadi huruf kecil semua.
import re
def casefolding(content):
    content = content.lower()
    content = content.strip(" ")
    content = re.sub(r'[?|$|.|!2_:")(-+,]','', content)
    return content
data['komen'] = data['komen'].apply(casefolding)
data.head(100)

,time,username,komen,rating,label
0,1 bulan lalu,rani,terimakasih untuk saler atas amanah nya semoga...,5,Positif
1,1 bulan lalu,Sofia,alhamdulillah sampai dengan selamat seller ram...,5,Positif
2,5 bulan lalu,M,hati yg mo beli pastikan brg sesuai dg kesepak...,1,Negatif
3,5 bulan lalu,Abdi,barang sdh sampai sesuai dgn deskripsi penjual...,5,Positif
4,5 bulan lalu,Ridho,untuk sejauh ini berjalan normal semoga bisa a...,4,Positif
...,...,...,...,...,...
95,1 bulan lalu,Daniel,tingkiu axioo barang yg diorder sesuai pesanan...,5,Positif
96,1 bulan lalu,M,semuanya aman hanya 1 yang kurang cepat panas ...,4,Positif
97,1 bulan lalu,Devia,pengiriman cepat sekalisemoga barangnya awet,4,Positif
98,1 bulan lalu,Haj,mantafff banget gan,5,Positif


In [ ]:
#proses tokenizing = metode ini dilakukan untuk pemisahan kata dalam suatu kalimat dengan tujuan untuk proses analisis teks lebih lanjut
def token(content):
    nstr = content.split(' ')
    dat= []
    a = -1
    for hu in nstr:
        a = a + 1
    if hu == '':
        dat.append(a)
    p = 0
    b = 0
    for q in dat:
        b = q - p
        del nstr[b]
        p = p + 1
    return nstr
data['komen'] = data['komen'].apply(token)
data.head(10)

,time,username,komen,rating,label
0,1 bulan lalu,rani,"[terimakasih, untuk, saler, atas, amanah, nya,...",5,Positif
1,1 bulan lalu,Sofia,"[alhamdulillah, sampai, dengan, selamat, selle...",5,Positif
2,5 bulan lalu,M,"[hati, yg, mo, beli, pastikan, brg, sesuai, dg...",1,Negatif
3,5 bulan lalu,Abdi,"[barang, sdh, sampai, sesuai, dgn, deskripsi, ...",5,Positif
4,5 bulan lalu,Ridho,"[untuk, sejauh, ini, berjalan, normal, semoga,...",4,Positif
5,6 bulan lalu,Purnama,"[laptop, tdk, bisa, charge]",2,Negatif
6,6 bulan lalu,Tria,"[sangat, memuaskan]",5,Positif
7,6 bulan lalu,Muji,"[penjual, bertanggung, jawab, laptop, sempat, ...",4,Positif
8,8 bulan lalu,Baduy,"[produk, sesuai, seller, ramah, packing, sm, p...",5,Positif
9,8 bulan lalu,Haidar,"[worth, it, dan, cukup, keren]",5,Positif


In [ ]:
#proses filtering = pada tahap ini kata-kata yang sudah melewati tahapa sebelumnya akan di proses untuk diambil kata kata penting dan membuan kata kata yang tidak teralu penting
import nltk #natural language toolkit
nltk.download('stopwords') # untuk menghapus kata kata tidak penting seperti yang, he, if
from nltk.corpus import stopwords

def stopword_removal(content):
    filtering = stopwords.words('indonesian','english')#pada bagian ini akan dilakukan penghapusan kata tidak perlu dalam bahasa indonesia maupun inggris
    x= []
    data = []
    def myFunc(x):
        if x in filtering : 
            return False
        else:
            return True
    fit = filter(myFunc, content)
    for x in fit:
        data.append(x)
        return data
data['komen'] = data['komen'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,time,username,komen,rating,label
0,1 bulan lalu,rani,[terimakasih],5,Positif
1,1 bulan lalu,Sofia,[alhamdulillah],5,Positif
2,5 bulan lalu,M,[hati],1,Negatif
3,5 bulan lalu,Abdi,[barang],5,Positif
4,5 bulan lalu,Ridho,[berjalan],4,Positif


In [ ]:
#proses steamming = pada bagian ini kata kata akan diolah dan di ambil kata dasanya saja seperti ditolong akan menjadi tolong
!pip install Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(content):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in content:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean= " ".join(do)
    print(d_clean)
    return d_clean
data['komen'] = data['komen'].apply(stemming)

data.to_csv('data_clean.csv',index=False)
data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 8.7 MB/s eta 0:00:00
terimakasih
alhamdulillah
hati
barang
jalan
laptop
muas
jual
produk
worth
fast
barang
moga
produk
produk
kerenbarang
seller
layan
barang
keren
sesuai
baik
mantap
masuk
moga
fungsi
mantap
barang
keren
sesuai
baik
mantap
nih
mantap
netbook
mantap
layar
laptop
layar
laptop
udah
barang
keyboard
barang
beli
barang
pakai
layar
drivernya
barang
price
dgn
seller
seeep
barang
ud
layar
sesuai
barang
huruf
blm
produk
barang
baterai
alhamdulillah
barang
baguuss
laptop
baterenya
fast
good
good
worth
mantaaaaap
bagus
okee
keyboard
barang
instal
baguss
good
good
barang
bentuk
moga
beli
barang
gila
hari
aman
good
bagus
katanaya
recommended
mantap
tingkiu
aman
kirim
mantafff
produk
alhamdulillah
cd
sesuai
barang
fast
asik
takut
mantap
pesan
anak
mantapp
paket
proses
pesan
kualitas
barang
kirim
ok
paket
produk
laptop
alhamduli

,time,username,komen,rating,label
0,1 bulan lalu,rani,terimakasih,5,Positif
1,1 bulan lalu,Sofia,alhamdulillah,5,Positif
2,5 bulan lalu,M,hati,1,Negatif
3,5 bulan lalu,Abdi,barang,5,Positif
4,5 bulan lalu,Ridho,jalan,4,Positif


In [ ]:
data_clean = data_clean.astype({'label': 'category'})
data_clean = data_clean.astype({'komen': 'string'})
data_clean.dtypes

time          object
username      object
komen         string
rating         int64
label       category
dtype: object

In [ ]:
#proses TF-IDF(Term Frequency Inverse Document Frequency) = pada metode yang digunakan untuk menentukan nilai frekuensi sebuah kata di dalam sebuah dokumen atau artikel dan juga frekuensi di dalam banyak dokumen
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

tvec = CountVectorizer()
X_cVec = tvec.fit_transform(data_clean['komen'].values.astype('U'))
print(X_cVec)
h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(X_cVec)
print(x_tfidf)
X = data_clean.komen
Y = data_clean.label


  (0, 108)	1
  (1, 1)	1
  (2, 35)	1
  (3, 12)	1
  (4, 39)	1
  (5, 55)	1
  (6, 71)	1
  (7, 43)	1
  (8, 89)	1
  (9, 117)	1
  (10, 29)	1
  (11, 12)	1
  (12, 69)	1
  (13, 89)	1
  (14, 89)	1
  (15, 47)	1
  (16, 101)	1
  (17, 56)	1
  (18, 12)	1
  (19, 46)	1
  (20, 103)	1
  (21, 11)	1
  (22, 63)	1
  (23, 67)	1
  (24, 69)	1
  :	:
  (294, 9)	1
  (295, 55)	1
  (296, 69)	1
  (297, 27)	1
  (298, 100)	1
  (299, 1)	1
  (300, 12)	1
  (301, 107)	1
  (302, 101)	1
  (303, 52)	1
  (304, 12)	1
  (305, 55)	1
  (306, 55)	1
  (307, 36)	1
  (308, 55)	1
  (309, 112)	1
  (310, 103)	1
  (311, 65)	1
  (312, 51)	1
  (313, 5)	1
  (314, 83)	1
  (315, 95)	1
  (316, 63)	1
  (317, 85)	1
  (318, 50)	1
  (0, 108)	1.0
  (1, 1)	1.0
  (2, 35)	1.0
  (3, 12)	1.0
  (4, 39)	1.0
  (5, 55)	1.0
  (6, 71)	1.0
  (7, 43)	1.0
  (8, 89)	1.0
  (9, 117)	1.0
  (10, 29)	1.0
  (11, 12)	1.0
  (12, 69)	1.0
  (13, 89)	1.0
  (14, 89)	1.0
  (15, 47)	1.0
  (16, 101)	1.0
  (17, 56)	1.0
  (18, 12)	1.0
  (19, 46)	1.0
  (20, 103)	1.0
  (21, 11)	1.0
 

In [ ]:
#Proses eksekusi algoritma KFold
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

kf = KFold(n_splits=10)
X_array = x_tfidf.toarray()
def cross_val(estimator):
  acc = []
  pcs = []
  rec = []

  for train_index, test_index in kf.split(X_array, Y):
    X_train, X_test = X_array[train_index], X_array[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    model = estimator.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc.append(accuracy_score(y_test, y_pred))
    pcs.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
    rec.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
    rec.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

    print(classification_report(y_test, y_pred, zero_division=0))
    print(f'Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}')
    print('=================================================\n')

  print(f'Average Akurasi: {np.mean(acc)}')
  print(f'Average Presisi: {np.mean(pcs)}')
  print(f'Average Recall: {np.mean(rec)}')
  print(f'Average F1-Score: {np.mean(rec)}')

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
cross_val(nb)

              precision    recall  f1-score   support

     Negatif       0.00      0.00      0.00         2
     Positif       0.94      1.00      0.97        30

    accuracy                           0.94        32
   macro avg       0.47      0.50      0.48        32
weighted avg       0.88      0.94      0.91        32

Confusion Matrix:
 [[ 0  2]
 [ 0 30]]

              precision    recall  f1-score   support

     Negatif       0.00      0.00      0.00        18
     Positif       0.44      1.00      0.61        14

    accuracy                           0.44        32
   macro avg       0.22      0.50      0.30        32
weighted avg       0.19      0.44      0.27        32

Confusion Matrix:
 [[ 0 18]
 [ 0 14]]

              precision    recall  f1-score   support

     Negatif       0.00      0.00      0.00         3
     Positif       0.91      1.00      0.95        29

    accuracy                           0.91        32
   macro avg       0.45      0.50      0.48       